In [19]:
import pandas as pd
import os
from pathlib import Path

absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "raw_datasets")
cleanedDataPath = os.path.join(absPath, "cleaned_datasets")
newDataPath = os.path.join(absPath, "new_data")



In [20]:
# saving folder
savePath = newDataPath

# parameters and URLs
gamesPath  = datasetsPath + "/games.csv"
vgchartzPath = datasetsPath + "/vgchartz-7_7_2020.csv"
indiegamesdevelopersPath = datasetsPath + "/indie-games-developers.csv"
developerPath = datasetsPath + "/video-games-developers.csv"


In [21]:
vgchartz = pd.read_csv(vgchartzPath)
games = pd.read_csv(gamesPath)
indiegamesdevelopers = pd.read_csv(indiegamesdevelopersPath)
developers = pd.read_csv(developerPath)

games.csv cleaning

In [22]:
# If not present the developer place it equal to the publishers in gemes.cvs
def isNaN(string):
    return string != string

for index, row in games.iterrows():
    if isNaN(row["developers"]) :
        games.at[index,"developers"]= row["publishers"]
        
games.drop(columns=['release_na', 'release_eu', 'release_jp'])
games['platforms']

0                                      PC
1                      Xbox 360, Xbox One
2        PC, PlayStation 2, PlayStation 4
3        PC, PlayStation 2, PlayStation 4
4        PC, PlayStation 2, PlayStation 4
                       ...               
35917                                 NaN
35918                                 NaN
35919                                 NaN
35920                                 NaN
35921                                 NaN
Name: platforms, Length: 35922, dtype: object

vgchartz Cleaning

In [23]:
vgchartz.astype({'vg_score': 'float'}).dtypes
vgchartz.astype({'critic_score': 'float'}).dtypes
vgchartz.astype({'user_score': 'float'}).dtypes
vgchartz.astype({'total_shipped': 'float'}).dtypes
vgchartz.astype({'total_sales': 'float'}).dtypes
vgchartz.astype({'na_sales': 'float'}).dtypes
vgchartz.astype({'jp_sales': 'float'}).dtypes
vgchartz.astype({'other_sales': 'float'}).dtypes

Unnamed: 0         int64
img               object
title             object
console           object
genre             object
publisher         object
developer         object
vg_score         float64
critic_score     float64
user_score       float64
total_shipped    float64
total_sales      float64
na_sales         float64
jp_sales         float64
pal_sales        float64
other_sales      float64
release_date      object
last_update       object
dtype: object

In [24]:
vgchartz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57950 entries, 0 to 57949
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     57950 non-null  int64  
 1   img            57950 non-null  object 
 2   title          57950 non-null  object 
 3   console        57950 non-null  object 
 4   genre          57950 non-null  object 
 5   publisher      57950 non-null  object 
 6   developer      57933 non-null  object 
 7   vg_score       984 non-null    float64
 8   critic_score   6544 non-null   float64
 9   user_score     386 non-null    float64
 10  total_shipped  2373 non-null   float64
 11  total_sales    19319 non-null  float64
 12  na_sales       12900 non-null  float64
 13  jp_sales       6993 non-null   float64
 14  pal_sales      13111 non-null  float64
 15  other_sales    15440 non-null  float64
 16  release_date   54296 non-null  object 
 17  last_update    11763 non-null  object 
dtypes: flo

In [25]:
# match the title with games.csv's title
vgchartz = pd.merge(vgchartz, games['title'], how ='inner', on =['title'])
#clean Series
vgchartz = vgchartz.drop(vgchartz[vgchartz.console =="Series"].index)
#drop unwanted columns
vgchartz.drop(columns=['Unnamed: 0','img', 'genre', 'publisher', 'developer', 'pal_sales', 'release_date', 'last_update'])


,title,console,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,other_sales
1,Devil May Cry,PS2,NaN,9.1,9.3,2.16,NaN,NaN,NaN,NaN
2,Devil May Cry,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Grand Theft Auto V,PS3,NaN,9.4,NaN,NaN,20.32,6.37,0.99,3.12
4,Grand Theft Auto V,PS4,NaN,9.7,NaN,NaN,19.39,6.06,0.60,3.02
5,Grand Theft Auto V,X360,NaN,NaN,NaN,NaN,15.86,9.06,0.06,1.42
...,...,...,...,...,...,...,...,...,...,...
26010,The 25th Ward: The Silver Case,PS4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26011,The 25th Ward: The Silver Case,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26012,The Mind's Eclipse,PC,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26013,Without Within,PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
vgchartzfilename = savePath +'vgchartz_cleaned.csv'
#vgchartz.to_csv(filename) 

**indie-games-developers_cleaning**

In [27]:
# match the developer with games.csv's developer
collect = games.groupby('developers')['title'].apply(list).reset_index(name='game_titles')
collect.rename(columns={'developers': 'Developer'}, inplace=True)
collect['Developer'] = collect['Developer'].apply(lambda x:x.lower())
indiegamesdevelopers['Developer'] = indiegamesdevelopers['Developer'].apply(lambda x:x.lower())

indiegamesdevelopers = indiegamesdevelopers.merge(collect, how ='inner', on =['Developer'])
indiegamesdevelopers = indiegamesdevelopers.drop(columns=['City', 'Autonomous area', 'Notes'])
indiegamesdevelopers = indiegamesdevelopers.dropna()


def match_title(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:    
    if w in Notable_games:
      matched_titles.append(w)
  indiegamesdevelopers['selected_titles'][ind] = matched_titles

def match_title_include_series(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:
    if 'series' in Notable_games :
      w_words=w.split()
      for item in w_words:
        if item in Notable_games:
          matched_titles.append(w)

    elif w in Notable_games:
      matched_titles.append(w)
      
  matched_titles = list(set(matched_titles)) 
  indiegamesdevelopers['selected_titles_series'][ind] = matched_titles


indiegamesdevelopers['selected_titles']=''
indiegamesdevelopers['selected_titles_series']=''
for ind in indiegamesdevelopers.index:
  match_title(ind, indiegamesdevelopers['game_titles'][ind], indiegamesdevelopers['Notable games'][ind])
  match_title_include_series(ind, indiegamesdevelopers['game_titles'][ind], indiegamesdevelopers['Notable games'][ind])

indiegamesdevelopers = indiegamesdevelopers.drop(columns=['game_titles'])



In [28]:
indiegamesdevelopers

,Developer,Country,Notable games,selected_titles,selected_titles_series
0,11 bit studios,Poland,AnomalyThis War of MineFrostpunk,[],[]
1,11 bit studios,Poland,AnomalyThis War of MineFrostpunk,"[This War of Mine, Frostpunk]","[Frostpunk, This War of Mine]"
2,ace team,Chile,Zeno ClashRock of Ages,"[Rock of Ages, Zeno Clash]","[Zeno Clash, Rock of Ages]"
3,alientrap,Canada,CapsizedApotheon,[Apotheon],[Apotheon]
4,ambrosia software,United States,Escape Velocity series,[Escape Velocity],"[Escape Velocity Override, Escape Velocity]"
...,...,...,...,...,...
154,zoetrope interactive,Turkey,Darkness WithinDarkness Within: The Dark Lineage,[],[]
155,zoink,Sweden,PlayStation All-Stars IslandThe Kore Gang,[],[]
156,zoonami,United Kingdom,ZendokuBonsai Barber,"[Bonsai Barber, Zendoku]","[Zendoku, Bonsai Barber]"
157,zootfly,Slovenia,First Battalion,[],[]


In [29]:
indiegamesdevelopersfilename = savePath +'indiegamesdevelopers_cleaned.csv'
#indiegamesdevelopers.to_csv(indiegamesdevelopersfilename) 

In [30]:
# indie selected_titles match
indiegamesdevelopers.selected_titles.str.len().sum()

179

In [31]:
# indie selected_titles_series match
indiegamesdevelopers.selected_titles_series.str.len().sum()

404

deleloper_cleaning

In [32]:
developers.rename(columns={'Notable games, series or franchises': 'Notable games'}, inplace=True)


# match the developer with games.csv's developer
collect = games.groupby('developers')['title'].apply(list).reset_index(name='game_titles')
#collect = games.groupby('developers')['id'].apply(list).reset_index(name='game_index')
collect.rename(columns={'developers': 'Developer'}, inplace=True)

collect['Developer'] = collect['Developer'].apply(lambda x:x.lower())
developers['Developer'] = developers['Developer'].apply(lambda x:x.lower())


developers = developers.merge(collect, how ='inner', on =['Developer'])
developers = developers.drop(columns=['City', 'Administrative division', 'Country','Est.','Notes'])
developers = developers.dropna()

def match_title(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:    
    if w in Notable_games:
      matched_titles.append(w)
  developers['selected_titles'][ind] = matched_titles

def match_title_include_series(ind, game_titles, Notable_games):
  matched_titles = []
  for w in game_titles:
    if 'series' in Notable_games :
      w_words=w.split()
      for item in w_words:
        if item in Notable_games:
          matched_titles.append(w)

    elif w in Notable_games:
      matched_titles.append(w)
      
  matched_titles = list(set(matched_titles)) 
  developers['selected_titles_series'][ind] = matched_titles

developers['selected_titles']=''
developers['selected_titles_series']=''
for ind in developers.index:
  match_title(ind, developers['game_titles'][ind], developers['Notable games'][ind])
  match_title_include_series(ind, developers['game_titles'][ind], developers['Notable games'][ind])


  
developers = developers.drop(columns=['game_titles'])

In [33]:
# selected_titles match
developers.selected_titles.str.len().sum()

520

In [34]:
# selected_titles_series match
developers.selected_titles_series.str.len().sum()

3154

In [35]:
developers

,Developer,Notable games,selected_titles,selected_titles_series
0,0verflow,School DaysSummer DaysCross Days,[],[]
1,11 bit studios,Frostpunk,[],[]
2,11 bit studios,Frostpunk,[Frostpunk],[Frostpunk]
3,1c company,King's Bounty: Warriors of the North,[],[]
4,2k czech,MafiaMafia II,[Mafia II],[Mafia II]
...,...,...,...,...
627,zipper interactive,SOCOM U.S. Navy SEALs seriesMechWarrior 3,[MechWarrior 3],"[SOCOM: U.S. Navy SEALs Combined Assault, Mech..."
628,zombie studios,Spec Ops: Rangers Lead the WayBlacklight: Retr...,[Blacklight: Retribution],[Blacklight: Retribution]
629,zootfly,Marlow Briggs and the Mask of Death,[],[]
630,zootfly,Marlow Briggs and the Mask of Death,[],[]


In [36]:
devgamesdevelopersfilename = savePath +'/indiegamesdevelopers_cleaned.csv'
#developers.to_csv(devgamesdevelopersfilename) 